In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

import warnings
warnings.filterwarnings('ignore')

In [2]:
file_path = 'Airplane_Crashes.csv'
df = pd.read_csv(file_path, low_memory=False)
df.head(5)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          5268 non-null   object 
 1   Time          3049 non-null   object 
 2   Location      5248 non-null   object 
 3   Operator      5250 non-null   object 
 4   Flight #      1069 non-null   object 
 5   Route         3561 non-null   object 
 6   Type          5241 non-null   object 
 7   Registration  4933 non-null   object 
 8   cn/In         4040 non-null   object 
 9   Aboard        5246 non-null   float64
 10  Fatalities    5256 non-null   float64
 11  Ground        5246 non-null   float64
 12  Summary       4878 non-null   object 
dtypes: float64(3), object(10)
memory usage: 535.2+ KB


In [4]:
df = df.dropna()

Искусственное создание целевой переменной

In [5]:
df['Fatal_Rate'] = df['Fatalities'] / df['Aboard']
df['HighFatality'] = (df['Fatal_Rate'] > 0.5).astype(int)

## Масштабирование признаков

Масштабирование — важный этап подготовки данных, особенно при использовании моделей, чувствительных к масштабу признаков (например, логистическая регрессия или kNN). В этой части мы применим:

StandardScaler — стандартизация до нормального распределения (среднее = 0, стандартное отклонение = 1);

MinMaxScaler — нормализация в диапазон [0, 1];

RobustScaler — масштабирование на основе медианы и межквартильного размаха, устойчивое к выбросам.

In [6]:
numeric_features = ['Aboard', 'Fatalities', 'Ground', 'Fatal_Rate']

scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

scaled_dfs = {}

for name, scaler in scalers.items():
    scaled_data = scaler.fit_transform(df[numeric_features])
    scaled_df = pd.DataFrame(scaled_data, columns=df[numeric_features].columns)
    scaled_dfs[name] = scaled_df
    print(f"\n{name}:")
    display(scaled_df.head())


StandardScaler:


,Aboard,Fatalities,Ground,Fatal_Rate
0,-0.660659,-0.481408,-0.052295,0.650414
1,-0.772765,-0.616545,-0.052295,0.650414
2,-0.814805,-0.667222,-0.052295,0.650414
3,-0.688686,-0.667222,-0.052295,-1.211415
4,-0.828819,-0.684114,-0.052295,0.650414



MinMaxScaler:


,Aboard,Fatalities,Ground,Fatal_Rate
0,0.023328,0.027444,0.0,1.000000
1,0.010886,0.013722,0.0,1.000000
2,0.006221,0.008576,0.0,1.000000
3,0.020218,0.008576,0.0,0.357143
4,0.004666,0.006861,0.0,1.000000



RobustScaler:


,Aboard,Fatalities,Ground,Fatal_Rate
0,-0.297297,-0.100000,0.0,0.000000
1,-0.405405,-0.245455,0.0,0.000000
2,-0.445946,-0.300000,0.0,0.000000
3,-0.324324,-0.300000,0.0,-1.607143
4,-0.459459,-0.318182,0.0,0.000000


## Обработка выбросов

Выбросы — это аномально большие или малые значения, которые могут искажать результаты моделей. Мы применим два подхода:

Удаление выбросов методом IQR: ;

Замена выбросов с использованием межквартильного размаха (IQR): выбросы заменяются на медиану по признаку.

**IQR (межквартильный размах)** — это разница между **третьим квартилем (Q3)** и **первым квартилем (Q1)**. Он показывает, где находится "середина" 50% данных.

$$
\text{IQR} = Q3 - Q1
$$

* **Q1 (25-й процентиль)** — значение, ниже которого находится 25% данных.
* **Q3 (75-й процентиль)** — значение, ниже которого находится 75% данных.

Любые значения, выходящие **сильно за пределы** IQR, считаются выбросами. Эти "пределы" вычисляются так:

$$
\text{Нижняя граница} = Q1 - 1.5 \times IQR
$$

$$
\text{Верхняя граница} = Q3 + 1.5 \times IQR
$$

Если значение меньше нижней границы или больше верхней — это выброс.


In [7]:
Q1 = df['Fatalities'].quantile(0.25)
Q3 = df['Fatalities'].quantile(0.75)
IQR = Q3 - Q1
df_no_outliers = df[(df['Fatalities'] >= Q1 - 1.5 * IQR) & (df['Fatalities'] <= Q3 + 1.5 * IQR)]

print("Удалено строк:", len(df) - len(df_no_outliers))
print("Оставшиеся данные:", df_no_outliers.shape)

Удалено строк: 55
Оставшиеся данные: (889, 15)


In [8]:
cdf = df.copy()

Q1 = cdf['Aboard'].quantile(0.25)
Q3 = cdf['Aboard'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
median = cdf['Aboard'].median()
cdf['Aboard'] = np.where((cdf['Aboard'] < lower) | (cdf['Aboard'] > upper), median, cdf['Aboard'])

cdf.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Fatal_Rate,HighFatality
208,01/19/1930,18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-AT-C Tri Motor,NC9689,5-AT-046,16.0,16.0,0.0,"While en route to Los Angeles, the pilot, flyi...",1.000000,1
236,03/31/1931,10:45,"Bazaar, Kansas",Trans Continental and Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,NC-999,1063,8.0,8.0,0.0,"Shortly after taking off from Kansas City, one...",1.000000,1
334,08/31/1934,23:42,"Amazonia, Missouri",Rapid Air Transport,6,Omaha - St. Joseph,Stinson SM-6000B,NC10809,5004,5.0,5.0,0.0,The plane crashed about 11 miles from St. Jose...,1.000000,1
354,05/06/1935,03:30,"Atlanta, Missouri",Trans Continental and Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas DC-2-112,NC13785,1295,14.0,5.0,0.0,The plane crashed while en route from Albuquer...,0.357143,0
365,08/14/1935,23:45,"Near Gilmer, Texas",Delta Air Lines,4,Dallas - Atlanta,Stinson Model A,NC14599,9103,4.0,4.0,0.0,Crashed 3 miles south of Gilmer. The outboard ...,1.000000,1


## Обработка нестандартного признака
Summary - является текстовым (строковым), и напрямую использовать его в численных моделях нельзя. Вместо этого извлечём из него информативные числовые признаки:

- Длина текста в символах;
- Количество слов;
- Средняя длина слова;
- Количество заглавных слов;
- Количество восклицательных знаков;

In [9]:
df['Summary'] = df['Summary'].fillna('')

df['Summary_char_count'] = df['Summary'].apply(len)
df['Summary_word_count'] = df['Summary'].apply(lambda x: len(x.split()))
df['Summary_avg_word_len'] = df['Summary'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0)
df['Summary_uppercase_words'] = df['Summary'].apply(lambda x: sum(1 for word in x.split() if word.isupper()))

df[['Summary', 'Summary_char_count', 'Summary_word_count', 'Summary_avg_word_len', 'Summary_uppercase_words']].head()

,Summary,Summary_char_count,Summary_word_count,Summary_avg_word_len,Summary_uppercase_words
208,"While en route to Los Angeles, the pilot, flyi...",287,52,4.538462,0
236,"Shortly after taking off from Kansas City, one...",392,59,5.661017,0
334,The plane crashed about 11 miles from St. Jose...,231,39,4.948718,0
354,The plane crashed while en route from Albuquer...,635,99,5.424242,1
365,Crashed 3 miles south of Gilmer. The outboard ...,197,33,5.000000,0


## Отбор признаков
Признаков в датасете много, но не все они влияют на результат. Используем три подхода:

- Filter method: SelectKBest (ANOVA F-test);
- Wrapper method: RFE с логистической регрессией;
- Embedded method: feature importance с RandomForest.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 944 entries, 208 to 5265
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date                     944 non-null    object 
 1   Time                     944 non-null    object 
 2   Location                 944 non-null    object 
 3   Operator                 944 non-null    object 
 4   Flight #                 944 non-null    object 
 5   Route                    944 non-null    object 
 6   Type                     944 non-null    object 
 7   Registration             944 non-null    object 
 8   cn/In                    944 non-null    object 
 9   Aboard                   944 non-null    float64
 10  Fatalities               944 non-null    float64
 11  Ground                   944 non-null    float64
 12  Summary                  944 non-null    object 
 13  Fatal_Rate               944 non-null    float64
 14  HighFatality             944

In [11]:
# Отберем числовые признаки
features = ['Aboard', 'Fatalities', 'Ground', 'Fatal_Rate', 'Summary_char_count', 'Summary_word_count']
X = df[features]
y = df['HighFatality']

Filter методы: Оценивают важность признаков независимо от модели, используя статистику (корреляцию, F-тест и т.п.).

SelectKBest (ANOVA F-test) - Оценивает каждый признак по отдельности с помощью ANOVA F-теста (анализ дисперсии).

Сравнивает, насколько хорошо признак разделяет классы целевой переменной. Чем выше значение F-статистики, тем полезнее признак.

In [12]:
selector_filter = SelectKBest(score_func=f_classif, k=2)
X_new_filter = selector_filter.fit_transform(X, y)
print("Отобранные признаки (Filter method):", [features[i] for i in selector_filter.get_support(indices=True)])

Отобранные признаки (Filter method): ['Fatalities', 'Fatal_Rate']


Wrapped методы: Используют модель обучения для оценки подмножеств признаков. Пробуют разные комбинации и выбирают лучшую.

Recursive Feature Elimination - Постепенно удаляет наименее важные признаки, обучая модель и оценивая её качество на каждом шаге. Строит модель (у нас — логистическая регрессия). Удаляет самый "бесполезный" признак. Повторяет, пока не останется нужное число признаков.

In [13]:
model = LogisticRegression(max_iter=1000)
rfe = RFE(model, n_features_to_select=2)
X_new_rfe = rfe.fit_transform(X, y)
print("Отобранные признаки (Wrapper method):", [features[i] for i in range(len(features)) if rfe.support_[i]])

Отобранные признаки (Wrapper method): ['Aboard', 'Fatalities']


Embedded методы: Отбор признаков происходит в процессе обучения модели, через встроенные механизмы (например, веса, регуляризацию или важность признаков).

Feature importance с RandomForest - Встроено в модель случайного леса — она определяет важность каждого признака во время обучения. Считает, насколько сильно каждый признак влияет на уменьшение неопределённости (например, уменьшение энтропии) при разбиении деревьев.

In [14]:
forest = RandomForestClassifier(random_state=42)
forest.fit(X, y)
importances = forest.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
print("Отобранные признаки (Embedded method):")
print(feature_importance_df.sort_values(by='Importance', ascending=False))

Отобранные признаки (Embedded method):
              Feature  Importance
3          Fatal_Rate    0.760698
1          Fatalities    0.160653
0              Aboard    0.059819
4  Summary_char_count    0.008964
5  Summary_word_count    0.007913
2              Ground    0.001954
